<a href="https://colab.research.google.com/github/Abhijeetkhade11/KaggleCompetitions/blob/main/house_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np
import pandas as pd


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
train_df.info()


In [ ]:
# Drop Unnecessary columns

train_df.drop(columns=['Id','Alley','MasVnrType','FireplaceQu','PoolQC','Fence','MiscFeature'],axis=0,inplace=True)

In [ ]:
train_df.info()

In [ ]:
col_empty = train_df.isnull().sum()[train_df.isnull().sum() > 0]
col_empty

In [ ]:
col_empty.info()

In [ ]:
col_empty

In [ ]:
# Fill missing values using average
col_empty_fill=['LotFrontage','MasVnrArea','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1'
                ,'BsmtFinType2','Electrical','GarageType','GarageYrBlt','GarageFinish','GarageQual',
               'GarageCond']

num_cols = train_df[col_empty_fill].select_dtypes(include='number').columns
cat_cols = train_df[col_empty_fill].select_dtypes(include='object').columns

In [ ]:
train_df[num_cols] = train_df[num_cols].fillna(train_df[num_cols].mean())


In [ ]:
train_df[cat_cols] = train_df[cat_cols].fillna(train_df[cat_cols].mode())

In [ ]:
train_df.info()

In [ ]:
train_df['SalePrice'].describe()

In [ ]:
sns.histplot(train_df['SalePrice'], bins=50, kde=True)

# Model Training

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

In [ ]:
y = train_df['SalePrice']
X = train_df.drop('SalePrice', axis=1)

# Feature Engineering
X['TotalSF'] = (
    X['TotalBsmtSF'] +
    X['1stFlrSF'] +
    X['2ndFlrSF']
)

X['HouseAge'] = X['YrSold'] - X['YearBuilt']
X['RemodAge'] = X['YrSold'] - X['YearRemodAdd']



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size= 0.2,
    random_state=42
)


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.impute import SimpleImputer

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

In [ ]:

num_cols = X.select_dtypes(include=['int64','float64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_imputer, num_cols),
        ('cat', Pipeline([
            ('imputer', cat_imputer),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_cols)
    ]
)



In [ ]:
model = Pipeline([
    ('preprocess', preprocessor),
    ('xgb', XGBRegressor(
        objective='reg:squarederror',
        n_estimators=2000,
        learning_rate=0.03,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ))
])

In [ ]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'RMSE: {rmse:.3f}')


In [ ]:

test_df['TotalSF'] = (
    test_df['TotalBsmtSF'] +
    test_df['1stFlrSF'] +
    test_df['2ndFlrSF']
)

test_df['HouseAge'] = test_df['YrSold'] - test_df['YearBuilt']
test_df['RemodAge'] = test_df['YrSold'] - test_df['YearRemodAdd']

In [ ]:
test_preds = np.expm1(model.predict(test_df))


In [ ]:
submission = pd.DataFrame({
    'Id': test_df['Id'],
    'SalePrice': test_preds
})

In [ ]:
submission.to_csv('submission.csv', index=False)
